## Steps

All steps that are played by default are **preprocessing** steps, but you can also add **postprocessing** steps, that act on the results instead of the circuit.

The steps will be filtered according to their type automatically.

In [ ]:
from pennylane_calculquebec.processing.config import MonarqDefaultConfig
from pennylane_calculquebec.processing.steps import IBUReadoutMitigation

readout_error_mitigation = IBUReadoutMitigation("yamaska")

my_config = MonarqDefaultConfig("yamaska")
my_config.steps.append(readout_error_mitigation)
print(*my_config.steps, sep="\n")

You can create new preprocessing / postprocessing steps by overriding the PreProcessing / PostProcessing classes.

In [3]:
# abstract steps + empty config
from pennylane_calculquebec.processing.interfaces import PreProcStep, PostProcStep
from pennylane_calculquebec.processing.config import ProcessingConfig

# default steps
from pennylane_calculquebec.processing.steps import CliffordTDecomposition, ASTAR, Swaps, IterativeCommuteAndMerge, MonarqDecomposition

In [4]:
# toy preprocessing step for printing the circuit operations
class PrintCircuit(PreProcStep):
    def execute(self, tape):
        print(*tape.operations)
        return tape
        
# toy postprocessing step for printing the results
class PrintResults(PostProcStep):
    def execute(self, tape, results):
        print(results)
        return results

In [ ]:
# this custom config will print the circuit, transpile, print the transpiled circuit 
# and then print the unmitigated results, followed by the mitigated results.
my_config = ProcessingConfig(PrintCircuit(),
                             CliffordTDecomposition(), 
                             ASTAR("yamaska"),
                             Swaps("yamaska"),
                             IterativeCommuteAndMerge(),
                             MonarqDecomposition(), 
                             PrintCircuit(),
                             PrintResults(),
                             IBUReadoutMitigation("yamaska"), 
                             PrintResults())
print(*my_config.steps, sep="\n")

### let's try our config with custom steps

In [ ]:
import pennylane as qml
from pennylane_calculquebec.API.client import MonarqClient

# Change the values in the parentheses for your credentials
my_client = MonarqClient("your host", "your user", "your access token", "your project")

dev = qml.device("monarq.default", client=my_client, processing_config = my_config, shots=1000)

dev.circuit_name = "your circuit"
dev.project = "your project"

# a simple ghz circuit
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1]) 
    qml.CNOT([1, 2])
    return qml.counts()

# lets print the circuit
print(qml.draw(circuit)())

results = circuit()

# you don't have to print results, since they are printed as a post processing step!
# print(results)

0: ──H─╭●────┤  Counts
1: ────╰X─╭●─┤  Counts
2: ───────╰X─┤  Counts
H(0) CNOT(wires=[0, 1]) CNOT(wires=[1, 2])
X90(wires=[1]) X90(wires=[0]) Z(0) CZ(wires=[4, 0]) X90(wires=[0]) X90(wires=[4]) Z(0) Z(4) CZ(wires=[0, 4]) X90(wires=[4]) X90(wires=[0]) Z(4) Z(0) CZ(wires=[4, 0]) X90(wires=[4]) Z(4) CZ(wires=[1, 4]) X90(wires=[4]) Z(4) CZ(wires=[4, 0]) X90(wires=[0]) X90(wires=[4]) Z(0) Z(4) CZ(wires=[0, 4]) X90(wires=[4]) X90(wires=[0]) Z(4) Z(0) CZ(wires=[4, 0]) X90(wires=[0]) X90(wires=[4]) Z(4) CZ(wires=[0, 4]) X90(wires=[4])
{'000': 428, '001': 51, '010': 26, '011': 60, '100': 75, '101': 75, '110': 72, '111': 213}
{'000': 465, '001': 38, '010': 5, '011': 56, '100': 37, '101': 64, '110': 44, '111': 290}
